In [2]:
# binary_classification 문제입니다.
# https://www.youtube.com/watch?v=PIjno6paszY&list=PLlMkM4tgfjnLSOjrEJN31gZATbcj_MpUm&index=11

# Classification

# Regression 은 숫자를 예측하는거라면
# Binary Classification 두개중에 하나를 고르는 거임!
# 스팸이냐? 혹은 햄?
# Facebook Show or Hide? 

In [1]:
# classification 알고리즘 어떻게 적용?
# linear regression 으로도 가능할거같다.
# 대략 0.5 되는 점 잡아서 크면 통과 작으면 불합격

# 예측하면 될것같다 
# 실제로 linear 쓸것이다 근데 문제가 있다.
# 근데 50시간 통과막 하면 1을 넘어가는 값이 나온다 
# 50 을 추가할시 선이 (0.5) 이상 이하의 직선의 기준이
# 달라진다.

# 그래서 linear Regression 은 간단하기는 하지만
# 0 과 1사이로 압축을 시켜주는 함수 하나 있으면 좋겠다.

# Sigmoid 를 써서 Cost 를 (y_hat-y) 제곱 형태로
# 경사 타면 이차함수 꼴이 안나오기에 새로운 cost 정의 필요

# https://www.youtube.com/watch?v=6vzchGYEJBc&list=PLlMkM4tgfjnLSOjrEJN31gZATbcj_MpUm&index=12

# cost 함수의미? : 실제값과 예측값이 비슷하면 cost 값은 작아지고
# 틀리면 cost 를 크게 해서 모델에 벌을 주는 것으로 의미

# -log(H(x))   => y=1 일때
# -log(1-H(x)) => y=0 일때

# 왜? exponential 과 상극인 log 사용
# -log(x) 함수는 0일떄 커지고 1일떄 작아짐
# H(x) 는 어떤 일이 일어날 확률

# -log(x) 와 -log(1-x)를 0과1사이에서 더할시
# 우리가 좋아하는 경사타고 내려가기 가능!

# C(H(x),y) = -ylog(H(x))-(1-y)log(1-H(x))

In [3]:
# 1. 직접 미분하면서 구해보기
import torch
import torch.nn.functional as F

# 5시간 이상 공부한 친구들 시험 합격
x_data = [[1],[2],[3],[6],[7],[8],[10],[0.5]]
y_data = [[0],[0],[0],[1],[1],[1],[1],[0]]

x_train = torch.FloatTensor(x_data)
y_train = torch.FloatTensor(y_data)

W = torch.zeros((1,1))
b = torch.zeros(1)

learning_rate = 0.01

for epoch in range(1000):
    hypothesis = torch.sigmoid(x_train@W +b)
    cost = -(
        y_train * torch.log(hypothesis) + (1-y_train)* torch.log(1-hypothesis)
    ).mean()
    # 미분시 결과값 : https://smwgood.tistory.com/6
    
    gradient_w = ((hypothesis - y_train)*x_train).mean()
    gradient_b = (hypothesis - y_train).mean()
    
    W -= learning_rate * gradient_w
    b -= learning_rate * gradient_b 
    
    if epoch % 100 == 0:
        print(f'Epoch ({str(epoch).rjust(4)}/1000) cost: {round(float(cost),6)}, W: {round(float(W),7)}, b:{round(float(b),7)}')
       
print('W: {:10f}'.format(float(W)))
print('b: {:10f}'.format(float(b)))
print('result : ')
print(torch.sigmoid(x_train * W + b))    

Epoch (   0/1000) cost: 0.693147, W: 0.0153125, b:0.0
Epoch ( 100/1000) cost: 0.492694, W: 0.2598578, b:-0.1834844
Epoch ( 200/1000) cost: 0.450955, W: 0.296653, b:-0.3842742
Epoch ( 300/1000) cost: 0.414819, W: 0.329512, b:-0.5713641
Epoch ( 400/1000) cost: 0.383442, W: 0.3605963, b:-0.7456194
Epoch ( 500/1000) cost: 0.356097, W: 0.3900384, b:-0.9082236
Epoch ( 600/1000) cost: 0.33216, W: 0.4179401, b:-1.0602956
Epoch ( 700/1000) cost: 0.311107, W: 0.444406, b:-1.202866
Epoch ( 800/1000) cost: 0.292497, W: 0.4695402, b:-1.3368645
Epoch ( 900/1000) cost: 0.275965, W: 0.4934425, b:-1.4631263
W:   0.515985
b:  -1.581234
result : 
tensor([[0.2563],
        [0.3660],
        [0.4917],
        [0.8198],
        [0.8840],
        [0.9273],
        [0.9728],
        [0.2103]])


In [44]:
# 2. pytorch autograd 사용하여 실제값 구하기

import torch
import torch.nn.functional as F

# 5시간 이상 공부한 친구들 시험 합격
x_data = [[1],[2],[3],[6],[7],[8],[10],[0.5]]
y_data = [[0],[0],[0],[1],[1],[1],[1],[0]]

x_train = torch.FloatTensor(x_data)
y_train = torch.FloatTensor(y_data)

w = torch.rand((1,1), requires_grad=True)
b = torch.rand(1, requires_grad=True)

learning_rate = 0.01

for epoch in range(1000):
    hypothesis = torch.sigmoid(x_train@w +b)
    cost = -(
        y_train * torch.log(hypothesis) + (1-y_train)* torch.log(1-hypothesis)
    ).mean()
    
    w.retain_grad()
    b.retain_grad()
    cost.backward()
    
    w = w - learning_rate * w.grad  # w 는 resassigned 됨
    b = b -  learning_rate * b.grad
    # 만일 w -= learning_rate * w.grad 했을시 ,
    # w 는 수정되므로 ,leaf_node error 가 나오게 된다. 
    if epoch % 100 == 0:
        print(f'Epoch ({str(epoch).rjust(4)}/1000) cost: {round(float(cost),6)}, W: {round(float(w),7)}, b:{round(float(b),7)}')
    
print('W: {:10f}'.format(float(w)))
print('b: {:10f}'.format(float(b)))
print('result : ')
print(torch.sigmoid(x_train@w +b))        

Epoch (   0/1000) cost: 0.747136, W: 0.230431, b:0.7487779
Epoch ( 100/1000) cost: 0.647581, W: 0.1568906, b:0.4660423
Epoch ( 200/1000) cost: 0.585123, W: 0.1949894, b:0.2192409
Epoch ( 300/1000) cost: 0.530934, W: 0.2329034, b:-0.010258
Epoch ( 400/1000) cost: 0.48404, W: 0.2689974, b:-0.2236102
Epoch ( 500/1000) cost: 0.443475, W: 0.3032565, b:-0.4219276
Epoch ( 600/1000) cost: 0.408332, W: 0.3357261, b:-0.6064179
Epoch ( 700/1000) cost: 0.377798, W: 0.3664809, b:-0.7783027
Epoch ( 800/1000) cost: 0.351165, W: 0.3956135, b:-0.9387617
Epoch ( 900/1000) cost: 0.327831, W: 0.4232261, b:-1.0888975
W:   0.449168
b:  -1.228356
result : 
tensor([[0.3145],
        [0.4182],
        [0.5298],
        [0.8125],
        [0.8717],
        [0.9141],
        [0.9631],
        [0.2682]], grad_fn=<SigmoidBackward0>)


In [51]:
# 3. Neural Network 로 gradient_descent 구현

import torch
import torch.nn as nn
import torch.optim as optim

class MyNeuralNetwork(nn.Module):
    def __init__(self):
        # python3 에서는 MyNeuralNetwork, self 인자 넣나 안넣나 똑같다.
        super(MyNeuralNetwork, self).__init__()
        self.layer=nn.Linear(in_features=1, out_features=1, bias=True)
        
    def forward(self, input):
        output = self.layer(input)
        output = torch.sigmoid(output)
        return output

model = MyNeuralNetwork().to("cpu")

input = torch.tensor([[1],[2],[3],[6],[7],[8],[10],[0.5]])
y = torch.tensor([[0],[0],[0],[1],[1],[1],[1],[0]], dtype=torch.float)

# model.parameter 에는  w, b 가 있으며 SGD 로 학습시킬 계획이다.
optimizer = optim.SGD(model.parameters(), lr=0.01)

epochs = 1000
loss_fn = nn.BCELoss()

for epoch in range(10000):
    
    # retain_grad와 
    optimizer.zero_grad()
    loss = loss_fn(model(input), y)
    loss.backward()
    optimizer.step()
    if epoch % 1000 == 0:
         print(f'Epoch ({str(epoch).rjust(4)}/1000) cost: {round(float(loss),6)}, W: {round(float(model.layer.weight),7)}, b:{round(float(model.layer.bias),7)}')


print('W: {:10f}'.format(float(model.layer.weight)))
print('b: {:10f}'.format(float(model.layer.bias)))
print('result : ')
print(model(input))    

        

Epoch (   0/1000) cost: 2.013475, W: -0.3309584, b:-0.9473956
Epoch (1000/1000) cost: 0.210401, W: 0.6076831, b:-2.0532207
Epoch (2000/1000) cost: 0.149039, W: 0.7605073, b:-2.8166595
Epoch (3000/1000) cost: 0.116821, W: 0.8741115, b:-3.3709104
Epoch (4000/1000) cost: 0.096863, W: 0.964787, b:-3.8074448
Epoch (5000/1000) cost: 0.083199, W: 1.0404938, b:-4.1687555
Epoch (6000/1000) cost: 0.073204, W: 1.1056569, b:-4.4778156
Epoch (7000/1000) cost: 0.065542, W: 1.1629813, b:-4.748414
Epoch (8000/1000) cost: 0.059461, W: 1.2142402, b:-4.9894757
Epoch (9000/1000) cost: 0.054505, W: 1.2606591, b:-5.2071085
W:   1.303081
b:  -5.405491
result : 
tensor([[0.0163],
        [0.0574],
        [0.1830],
        [0.9178],
        [0.9762],
        [0.9934],
        [0.9995],
        [0.0085]], grad_fn=<SigmoidBackward0>)
